In [4]:

# -*- coding: UTF-8 -*-
__author__ = 'zy'
__time__ = '2020/7/25 23:08'
import requests
import math

key = '8VUjfqGwgnHwEZLxwPpnZvO1Sgeq2HFO'  # 这里填写你的百度开放平台的key
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率


def geocode(address):
    """
    利用百度geocoding服务解析地址获取位置坐标
    :param address:需要解析的地址
    :return:
    """
    geocoding = {'s': 'rsv3',
                 'key': key,
                 'city': '全国',
                 'address': address}
    res = requests.get(
        "http://restapi.amap.com/v3/geocode/geo", params=geocoding)
    if res.status_code == 200:
        json = res.json()
        status = json.get('status')
        count = json.get('count')
        if status == '1' and int(count) >= 1:
            geocodes = json.get('geocodes')[0]
            lng = float(geocodes.get('location').split(',')[0])
            lat = float(geocodes.get('location').split(',')[1])
            return [lng, lat]
        else:
            return None
    else:
        return None


def gcj02tobd09(lng, lat):
    """
    火星坐标系(GCJ-02)转百度坐标系(BD-09)
    谷歌、高德——>百度
    :param lng:火星坐标经度
    :param lat:火星坐标纬度
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]


def bd09togcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]


def wgs84togcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]


def gcj02towgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]


def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret


def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    if lng < 72.004 or lng > 137.8347:
        return True
    if lat < 0.8293 or lat > 55.8271:
        return True
    return False

def bd09towgs84(lng,lat):#114.277591,30.580842
    result2 = bd09togcj02(lng,lat)
    result4 = gcj02towgs84(result2[0], result2[1])
    return result4

import json,time
import pandas as pd
from urllib.request import urlopen, quote

def getlnglat(address):
    # http://api.map.baidu.com/geocoder/v2/?address=北京市海淀区上地十街10号&output=json&ak=您的ak&callback=showLocation //GET请求
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'
    ak = '8399wyuCP00HcW0c7rHfjMpCIT6dA4mC'  # 浏览器端密钥
    address = quote(address)  # 由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + address + '&output=' + output + '&ak=' + ak
    print(uri)
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    lat = temp['result']['location']['lat']
    lng = temp['result']['location']['lng']

    code = temp['status']
    # print(lat, lng,code,)
    return lat, lng

def get_wgs(area):
    # latlng = ('武汉·汉口火车站 ')
    geocode = getlnglat(area)
    # geocode =(39.93410210068737, 116.14756198146019)
    return bd09towgs84(geocode[1],geocode[0])
    #[116.13488415043483, 39.926555169915595]



In [5]:

area='武汉·汉口火车站 '
print(get_wgs(area))


http://api.map.baidu.com/geocoder/v2/?address=%E6%AD%A6%E6%B1%89%C2%B7%E6%B1%89%E5%8F%A3%E7%81%AB%E8%BD%A6%E7%AB%99%20&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
[116.13488415043483, 39.926555169915595]


In [10]:
with open('area.txt','r',encoding='utf-8') as f:
    areas=f.read()
print(areas.split('\n')[0:10])
areas=areas.split('\n')

['硚口区丰顺路至园博大道涵洞', '汉阳区自力二街鹦鹉小道', '武昌区紫阳东路东安路口', '洪山路东一路口', '公正路省图书馆门口', '友谊大道长江隧道入口', '小东门路口民主路右转上中山路渠化道', '梅苑路中南路街道办事处门口', '书卷路公正路路口', '中华路粮食局']


In [12]:
len(areas)



2990

In [15]:
data={"硚口区丰顺路至园博大道涵洞":[1,2]}

datas={}

In [ ]:
import time
latlngs=[]
for i in areas:
    if i not in datas:
        try:
            latlng=get_wgs(i)
            datas['i']=latlng
            time.sleep(0.05)
        except:
            time.sleep(5)
            try:
                latlng=get_wgs(i)
                datas['i']=latlng
            except:
                latlng=[0,0]
                datas['i']=latlng

    else:
        latlng=datas['i']
    latlngs.append(latlng)
with open('经纬度.txt','w',encoding='utf-8') as f:
    for i in range(len(latlngs)):
        f.write(areas[i]+','+str(latlngs[i][0])+','+
                str(latlngs[i][1]))

http://api.map.baidu.com/geocoder/v2/?address=%E7%A1%9A%E5%8F%A3%E5%8C%BA%E4%B8%B0%E9%A1%BA%E8%B7%AF%E8%87%B3%E5%9B%AD%E5%8D%9A%E5%A4%A7%E9%81%93%E6%B6%B5%E6%B4%9E&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
http://api.map.baidu.com/geocoder/v2/?address=%E6%B1%89%E9%98%B3%E5%8C%BA%E8%87%AA%E5%8A%9B%E4%BA%8C%E8%A1%97%E9%B9%A6%E9%B9%89%E5%B0%8F%E9%81%93&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
http://api.map.baidu.com/geocoder/v2/?address=%E6%B1%89%E9%98%B3%E5%8C%BA%E8%87%AA%E5%8A%9B%E4%BA%8C%E8%A1%97%E9%B9%A6%E9%B9%89%E5%B0%8F%E9%81%93&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
http://api.map.baidu.com/geocoder/v2/?address=%E6%AD%A6%E6%98%8C%E5%8C%BA%E7%B4%AB%E9%98%B3%E4%B8%9C%E8%B7%AF%E4%B8%9C%E5%AE%89%E8%B7%AF%E5%8F%A3&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
http://api.map.baidu.com/geocoder/v2/?address=%E6%B4%AA%E5%B1%B1%E8%B7%AF%E4%B8%9C%E4%B8%80%E8%B7%AF%E5%8F%A3&output=json&ak=8399wyuCP00HcW0c7rHfjMpCIT6dA4mC
http://api.map.baidu.com/geocoder/v2/?address=%E